## Imports

In [1]:
import pandas as pd
import numpy as np
import csv
import os
from tqdm import tqdm
import warnings
from ast import literal_eval
from string import ascii_letters

## Global Variables and Setup

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
path = os.getcwd() + "/data"
dirs = os.listdir(path)

In [4]:
final = ""

## Helper Function

In [5]:
def concat(source, targetCsvFile, product_type=""):
    global final
    print(source)
    df = pd.read_csv(targetCsvFile)
    for col in df.columns:
        name = col.lower()
        df.rename(columns={col : name}, inplace=True)
        
    df["product_type"] = product_type
    df["source"] = source
    
    if isinstance(final, str):
        final = df
    else:
        final = pd.concat([final, df], axis=0, ignore_index=True)

def explore_dir(source, curr_path):
    #Items in a source folder are either csv files or files representing the type of product
    for item in os.listdir(curr_path):
        item_in_source_path = os.path.join(curr_path, item)
        
        #If item is not ipynb and it is not a file aka it is a directory
        if not os.path.isfile(item_in_source_path) and ".ipynb" not in item_in_source_path:
            product_type = item
            path_to_files_in_product = item_in_source_path
            
            products = os.listdir(path_to_files_in_product)
            for product in products:
                #convert here and check item is not a jupyter checkpoint
                if ".ipynb" not in product:
                    concat(source, os.path.join(path_to_files_in_product, product), product_type=product_type)
        else:
            #This means the items in the source folder are files and we should convert them
            #convert here
            if ".ipynb" not in item:
                concat(source, os.path.join(curr_path,item))
                
def clean_item(cell):
    if cell is np.nan:
        return np.nan
    
    try:
        cell = literal_eval(cell)
        
        if isinstance(cell, list):
            return cell
    except:
        pass
    
    if isinstance(cell, str):
        #print(cell.split(","))
        return cell.split(",")
    else:
        return list(cell)

def flatten(cell):
    if isinstance(cell, list):
        new_list = []

        for item in cell:
            if isinstance(item, list):
                for i in item:
                    if i is not np.nan and i != "nan":
                        new_list.append(i)
            else:
                if item is not np.nan and item != "nan":
                    new_list.append(item)
        return new_list
    else:
        return cell
    
def replace_list_nan(cell):
    if isinstance(cell, list):
        all_nan = True
        for item in cell:
            if item is not np.nan:
                all_nan = False
                break
        if all_nan:
            return np.nan
        else:
            return cell
    else:
        return cell
    
def check_column(column_name):
    global final
    for item in final[final[column_name].notna()][column_name]:
        print(item)

def get_columns():
    global final
    return final.columns

# Data Processing

### Joining All Csv files

In [6]:
#Top level, finds the source of what each individual thing looks for
for source in tqdm(dirs):
    curr_path = os.path.join(path, source)
    
    if ".ipynb" not in curr_path:
        if not os.path.isfile(curr_path):
            explore_dir(source, curr_path)
            print()

 20%|████████████████▌                                                                  | 2/10 [00:00<00:00, 14.07it/s]C:\Users\wli423\AppData\Local\Temp\ipykernel_29396\1449192373.py:15: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  final = pd.concat([final, df], axis=0, ignore_index=True)


csvape

elementvape
elementvape
elementvape
elementvape
elementvape
elementvape
elementvape
elementvape
elementvape
elementvape
elementvape

getpop

mipod


 50%|█████████████████████████████████████████▌                                         | 5/10 [00:00<00:00, 20.96it/s]


myvaporstore

perfectvape

vapedotcom


 80%|██████████████████████████████████████████████████████████████████▍                | 8/10 [00:00<00:00, 14.73it/s]


vapesourcing

vapewh

vapingdotcom


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 15.72it/s]

### Cleaning joined dataframe in preperation for merging similar columns

In [7]:
final.replace('[]', np.nan, inplace=True)
final = final.loc[:, final.any()]

### Getting the conversions csv into a dictionary

In [8]:
#get conversions to prep
reader = csv.reader(open('conversions.csv', encoding='utf-8-sig'))

conversions = {}
for row in reader:
    key = row[0].strip()
    if key in conversions:
        # implement your duplicate row handling here
        pass
    conversions[key] = [x.strip() for x in row[1].split(',')]

### Merge Columns and rename. Drop old columns

In [9]:
with warnings.catch_warnings():
    for key, value in tqdm(conversions.items()):
        print(key)
        for old in value:
            try:
                final[old] = final[old].apply(clean_item)
            except:
                pass

        try:
            my_list = final[conversions[key]].values.tolist()
            final[key] = my_list

            for item in conversions[key]:
                if item not in conversions.keys():
                    final.drop(item, axis=1, inplace=True)
        except:
            print("Error with: ", key)

  0%|                                                                                           | 0/54 [00:00<?, ?it/s]<unknown>:13: SyntaxWarning: invalid decimal literal
<unknown>:14: SyntaxWarning: invalid decimal literal
<unknown>:15: SyntaxWarning: invalid decimal literal
  4%|███                                                                                | 2/54 [00:00<00:03, 13.23it/s]

adjustable_airflow_bool
adjustable_airflow
has_battery
battery_description


  7%|██████▏                                                                            | 4/54 [00:00<00:04, 10.87it/s]

bottle_capacity_ml
brand


 13%|██████████▊                                                                        | 7/54 [00:00<00:05,  8.03it/s]

...

<unknown>:1: SyntaxWarning: invalid decimal literal
 91%|██████████████████████████████████████████████████████████████████████████▍       | 49/54 [00:05<00:00,  8.37it/s]

product_code
product_id
puff_count
rechargeable_bool


 98%|████████████████████████████████████████████████████████████████████████████████▍ | 53/54 [00:05<00:00, 10.82it/s]

smart_led_bool
usb_bool
warnings


100%|██████████████████████████████████████████████████████████████████████████████████| 54/54 [00:05<00:00,  9.07it/s]


### Cleaning merged dataframe 

In [10]:
#Unroll all nested lists and replace all lists with all nan
for col in final.columns:    
    final[col] = final[col].apply(flatten)
    final[col] = final[col].apply(replace_list_nan)

#Turn prices into decimals
def turn_to_decimal(cell):
    if isinstance(cell, list):
        temp = []
        for item in cell:
            if isinstance(item, str):
                item = item.replace("From ", "")
                temp.append(item.replace("$", ""))
            else:
                temp.append(item)
        return temp
    else:
        if isinstance(cell, str):
            cell.replace("From ", "").replace("$", "")
            return [literal_eval(cell)]
        return cell
final["price"] = final["price"].apply(turn_to_decimal)

#replace \n with commas in flavors
def replace_commas_in_flavors(cell):
    temp = []
    
    try:
        for item in cell:
            temp.append(item.replace("\n", ", "))
        return temp
    except:
        return cell

final["flavors"] = final["flavors"].apply(replace_commas_in_flavors)

### Play around with final result and write it to a csv file

In [11]:
final.to_csv("merged_data.csv", encoding='utf-8', index=False)

In [33]:
check_column("nicotine_values")

['50mg']
['50mg']
['50mg']
['50mg']
['50mg', '20mg', '0mg']
['50mg']
['50mg']
['50mg']
['50mg', '20mg']
['50mg']
['50mg']
['50mg']
['50mg']
['0mg']
['50mg']
['3mg', '6mg', '8mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg', '20mg']
['50mg', '20mg']
['50mg', '20mg']
['50mg']
['50mg']
['50mg', '20mg', '0mg']
['50mg', '20mg', '0mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['40mg']
['40mg']
['40mg', '0mg']
['50mg']
['3mg', '6mg']
['3mg', '6mg']
['3mg', '6mg']
['3mg', '6mg']
['50mg']
['50mg']
['50mg']
['6mg', '8mg', '3mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg', '0mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['40mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50mg']
['50m